In [0]:
storage_account_name = "sfhexastorage"
container_name = "bronze"
account_key = "jBmM0OVBS0I8tvj1LLFb0qzJVIFk9+Qd/hNW0ZvNe6TiSbatHvHC6NoeSYm3nfbmy1hocQuVbnhw+AStRqeGIA=="

# Configure Spark
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    account_key
)

file_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/Customer.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)


+---+--------+---------+------+
| ID|CUSTOMER|   REGION|AMOUNT|
+---+--------+---------+------+
|  1|   Pooja|SRI LANKA|  1000|
|  2|Sivasree|    INDIA|  9999|
|  3|  Sakthi|       UK|  8888|
|  4|     Abi|      USA|  7777|
|  5|   Priya|       US|  6666|
+---+--------+---------+------+



In [0]:
options = {
    "sfURL": "sqishot-fa68768.snowflakecomputing.com",
    "sfDatabase": "PRACTICE",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "COMPUTE_WH",
    "sfRole": "ACCOUNTADMIN",
    "sfUser": "poojashree",
    "sfPassword": "Poojashree@307"
}
df = spark.read.format("snowflake").options(**options).option("dbtable", "Customer").load()
df.show()

+---+--------+---------+------+
| ID|CUSTOMER|   REGION|AMOUNT|
+---+--------+---------+------+
|  1|   Pooja|SRI LANKA|  1000|
|  2|Sivasree|    INDIA|  9999|
|  3|  Sakthi|       UK|  8888|
|  4|     Abi|      USA|  7777|
|  5|   Priya|       US|  6666|
|  1|   Pooja|SRI LANKA|  1000|
|  2|Sivasree|    INDIA|  9999|
|  3|  Sakthi|       UK|  8888|
|  4|     Abi|      USA|  7777|
|  5|   Priya|       US|  6666|
+---+--------+---------+------+



In [0]:
transformed_df = df.groupBy("Region").agg({"amount": "sum"}).withColumnRenamed("sum(amount)", "total_sales")

transformed_df.show(5)

+---------+-----------+
|   Region|total_sales|
+---------+-----------+
|SRI LANKA|       1000|
|       US|       6666|
|      USA|       7777|
|       UK|       8888|
|    INDIA|       9999|
+---------+-----------+



In [0]:
# Write to Snowflake
(transformed_df.write
  .format("snowflake")
  .options(**options)
  .option("dbtable", "Customer")
  .mode("overwrite")
  .save())
  
print("successfull")

successfull


In [0]:
%pip install snowflake-snowpark-python

In [0]:
%restart_python

In [0]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, avg

# Connection
connection_parameters = {
  "account": "sqishot-fa68768",
  "user":"poojashree",
  "password": "Poojashree@307",
  "warehouse": "COMPUTE_WH",
  "database": "PRACTICE",
  "schema": "PUBLIC"
}
session = Session.builder.configs(connection_parameters).create()

# Load Snowflake table
sales_df = session.table("Customer")

# Transformation inside Snowflake
region_sales = (sales_df
    .group_by(col("REGION"))
    .agg(avg(col("AMOUNT")).alias("AVG_SALES"))
)

region_sales.show()

---------------------------
|"REGION"   |"AVG_SALES"  |
---------------------------
|UK         |8888.000000  |
|US         |6666.000000  |
|SRI LANKA  |1000.000000  |
|USA        |7777.000000  |
|INDIA      |9999.000000  |
---------------------------



In [0]:
sales_df.show(5)
sales_df.printSchema()
region_sales.show()


--------------------------------------------
|"ID"  |"CUSTOMER"  |"REGION"   |"AMOUNT"  |
--------------------------------------------
|1     |Pooja       |SRI LANKA  |1000      |
|2     |Sivasree    |INDIA      |9999      |
|3     |Sakthi      |UK         |8888      |
|4     |Abi         |USA        |7777      |
|5     |Priya       |US         |6666      |
--------------------------------------------

root
 |-- "ID": StringType(16777216) (nullable = True)
 |-- "CUSTOMER": StringType(16777216) (nullable = True)
 |-- "REGION": StringType(16777216) (nullable = True)
 |-- "AMOUNT": LongType() (nullable = True)
---------------------------
|"REGION"   |"AVG_SALES"  |
---------------------------
|UK         |8888.000000  |
|US         |6666.000000  |
|SRI LANKA  |1000.000000  |
|USA        |7777.000000  |
|INDIA      |9999.000000  |
---------------------------

